In [2]:
#Installing the necessary libraries
!pip install requests
!pip install beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 435.7 kB/s eta 0:00:05
    --------------------------------------- 0.0/1.8 MB 388.9 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.8 MB 581.0 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.8 MB 774.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 827.9 kB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.8 MB 907.8 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 916.0 kB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.8 MB 955.7 kB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.8 MB 956.1 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.8 MB 992.4 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.8 MB 992.3 kB/s eta 0:00:02
   -

ERROR: To modify pip, please run the following command:
C:\Users\anoop\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#Importing the library functionalities

import requests
from bs4 import BeautifulSoup
import time
import urllib.robotparser

In [7]:
# Function to fetch the HTML content of a page
def fetch_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to fetch {url}: Status code {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

In [8]:
# Function to extract all links from a webpage
def extract_links(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = set()  # Using a set to avoid duplicate URLs
    for anchor in soup.find_all('a', href=True):
        href = anchor['href']
        if href.startswith('http'):  # Ensure it's a complete URL
            links.add(href)
        else:
            links.add(base_url + href)  # Handle relative URLs
    return links

In [9]:
# Main crawler function
def crawl(seed_url, max_pages=10):
    to_crawl = [seed_url]  # List of pages to visit
    crawled = set()  # Set of pages already visited

    while to_crawl and len(crawled) < max_pages:
        url = to_crawl.pop(0)  # Get the next page to crawl
        if url in crawled:
            continue  # Skip if already crawled

        print(f"Crawling: {url}")
        html_content = fetch_page(url)

        if html_content:
            links = extract_links(html_content, url)
            to_crawl.extend(links - crawled)  # Add new links to crawl
            crawled.add(url)

        time.sleep(1)  # Add a delay to avoid overwhelming servers

    print(f"Crawled {len(crawled)} pages")

# Start crawling from a seed URL
seed_url = "https://en.wikipedia.org"
crawl(seed_url)

Crawling: https://en.wikipedia.org
Crawling: https://ja.wikipedia.org/wiki/
Crawling: https://en.wikipedia.org/wiki/Markham%27s_storm_petrel
Crawling: https://en.wikipedia.org/wiki/Castro_District,_San_Francisco
Crawling: https://vi.wikipedia.org/wiki/
Crawling: https://en.wikipedia.org/wiki/Kathmandu
Crawling: https://pa.wikipedia.org/wiki/
Crawling: https://en.wikipedia.org/wiki/Bulgogi
Crawling: https://sh.wikipedia.org/wiki/
Crawling: https://en.wikipedia.org/wiki/Secretum_(British_Museum)
Crawled 10 pages
